# Steam Reviews 2021

## Table of Contents
<ol>
<li><a href="#intro"><strong>Introduction</strong></a></li>
    <ul>
        <li><a href="#Dictionary">Data Dictionary</a></li>
    </ul>
<br>
<li><a href="#Wrangling"><strong>Data Wrangling</strong></a></li>
    <ul>
        <li><a href="#Gather">Gather</a></li>
    </ul>
    <ul>
        <li><a href="#Assess">Assess</a></li>
            <ul>
                <li><a href="#Quality">Quality issues</a></li>
            </ul>
            <ul>
                <li><a href="#Tidiness">Tidiness Issues</a></li>
            </ul>
    </ul>
    <ul>
        <li><a href="#Clean">Clean</a></li>
            <ul>
                <li><a href="#Define">Define</a></li>
            </ul>
            <ul>
                <li><a href="#Code">Code</a></li>
            </ul>
            <ul>
                <li><a href="#Test">Test</a></li>
            </ul>
    </ul>

<a id='intro'></a>
## 1. Introduction

This notebook is a side notebook for the Data Wrangling part of our dataset. We will explore, assess, and clean the original dataset, and export the cleaned version to be used directly in our main notebook.

<a id='Dictionary'></a>
### Data Dictionary
Here's a describtion for the attributes:
* What game was the review written for: app_id, app_name ;
* in what language was the review: language ;
* whether it recommends or not recommended;
* information about the review: review_id, review;
* the time review was written and the time it was updated: timestamp_created, timestamp_updated
* how useful the review was: votes_helpful, votes_funny, weighted_vote_score;
* how many comments (replies) were written on the review comment_count
* whether the author bought the game or received it for free: steam_purchase,received_for_free;
* whether the author wrote a review when the game was in pre-release: written_during_early_access.

* Information about the Author (steam account): their identification key: author.steamid;
* how many games he has: author.num_games_owned;
* how many reviews he wrote: author.num_reviews;
* how many hours he played in total: author.playtime_forever;
* whether he played in the last two weeks: author.playtime_last_two_weeks;
* how long he played the game to write a review: author.playtime_at_review;
* the date of his last game: author.last_played;

<a id='Wrangling'></a>
## 2. Data Wrangling
<a id='Gather'></a>
### a) Gathering the Data

Importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from datetime import datetime

Since the data is already ready on Kaggle and we don't need to scrap it neither download it from Steam API, we will just download it directly from Kaggle.

However, the size of data is 8 gigs so we have to either import only the necessary columns of or load it batch by batch. For the sake of first  exploration, I'll load all attributes except of review attribute which holds the string review of each entry. Such attribute takes so much unecessary storage that we don't need at all.

Now, we can work on just 3 gigs of storage for the entire dataset. After, analyzing the data and define the major issues and clean them. I will work only on the necessary attributes to address each question in our EDA instead of working with the whole data including the unncessary attributes.

I expect to spend a bunch of time analyzing the data and cleaning it properly, and spend much less time covering our EDA.

---

Reading the dataset columns.

In [2]:
cols = list(pd.read_csv("steam_reviews.csv", nrows =1))
print(cols)

['Unnamed: 0', 'app_id', 'app_name', 'review_id', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'recommended', 'votes_helpful', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'author.steamid', 'author.num_games_owned', 'author.num_reviews', 'author.playtime_forever', 'author.playtime_last_two_weeks', 'author.playtime_at_review', 'author.last_played']


Localizing the index of the review attribute programatically.

In [3]:
review_col_idx = cols.index('review')
review_col_idx

5

Loading all the attributes except for the review in our dataframe.

In [4]:
df = pd.read_csv('steam_reviews.csv', #nrows=5, 
                 usecols=[i for i in range(1, len(cols)) if i!=review_col_idx])

Let's get an idea about the rows, columns, names and length.

In [5]:
df.columns

Index(['app_id', 'app_name', 'review_id', 'language', 'timestamp_created',
       'timestamp_updated', 'recommended', 'votes_helpful', 'votes_funny',
       'weighted_vote_score', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access', 'author.steamid',
       'author.num_games_owned', 'author.num_reviews',
       'author.playtime_forever', 'author.playtime_last_two_weeks',
       'author.playtime_at_review', 'author.last_played'],
      dtype='object')

In [6]:
display(df.shape)
display(df.head(2))
df.tail(2)

(21747371, 21)

,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,1611381629,1611381629,True,0,0,0.0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1.611343e+09
1,292030,The Witcher 3: Wild Hunt,85185250,schinese,1611381030,1611381030,True,0,0,0.0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1.611386e+09


,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
21747369,546560,Half-Life: Alyx,65645066,english,1584986631,1586382422,True,0,0,0.386022,...,True,False,False,76561198041763187,140,3,210.0,0.0,12.0,1.589715e+09
21747370,546560,Half-Life: Alyx,65644930,schinese,1584986505,1592060800,True,0,0,0.416107,...,True,False,False,76561198116747069,253,7,1062.0,0.0,8.0,1.602858e+09


Let's get an idea about the timestamp.

In [7]:
pd.to_datetime(df.timestamp_updated[:10], origin='unix', unit='s')

0   2021-01-23 06:00:29
1   2021-01-23 05:50:30
2   2021-01-23 05:46:40
3   2021-01-23 05:32:50
4   2021-01-23 05:23:47
5   2021-01-23 05:21:04
6   2021-01-23 05:18:11
7   2021-01-23 05:05:12
8   2021-01-23 04:55:03
9   2021-01-23 04:43:25
Name: timestamp_updated, dtype: datetime64[ns]

Let's display the number of duplicated entries.

In [8]:
df.duplicated().sum()

134927

Let's make sure that there are no more duplicated entries by review_id.

In [9]:
df.review_id.duplicated().sum()

134927

In [10]:
(df.duplicated() != df.review_id.duplicated()).sum()

0

Well, then the duplicated entries are already duplicated by review_id so we have a total of 134927 duplicated reviews not more.

---

Now, let's display some duplicated entries to make sure of it.

In [11]:
df[df.review_id.duplicated()].head()

,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
21471076,367520,Hollow Knight,84774500,english,1610783706,1610783706,True,0,0,0.0,...,True,False,False,76561198831230953,11,7,262.0,232.0,242.0,1.610785e+09
21471077,367520,Hollow Knight,84774347,koreana,1610783377,1610783377,True,0,0,0.0,...,True,False,False,76561198282451594,38,9,5425.0,9.0,5425.0,1.610374e+09
21471078,367520,Hollow Knight,84774076,spanish,1610782931,1610782931,True,0,0,0.0,...,True,False,False,76561198322215609,8,3,1195.0,0.0,1195.0,1.609371e+09
21471079,367520,Hollow Knight,84773959,schinese,1610782697,1610782697,True,0,0,0.0,...,True,False,False,76561198090497424,198,53,2440.0,1510.0,2440.0,1.610773e+09
21471080,367520,Hollow Knight,84773690,english,1610782249,1610782249,True,0,0,0.0,...,False,False,False,76561198150429807,15,2,2405.0,1839.0,2314.0,1.610788e+09


In [12]:
df[df.review_id == 84774500]

,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
3609983,367520,Hollow Knight,84774500,english,1610783706,1610783706,True,0,0,0.0,...,True,False,False,76561198831230953,11,7,262.0,232.0,242.0,1.610785e+09
21471076,367520,Hollow Knight,84774500,english,1610783706,1610783706,True,0,0,0.0,...,True,False,False,76561198831230953,11,7,262.0,232.0,242.0,1.610785e+09


---
Let's get an idea about the value uniqueness of each attribute/feature.

In [13]:
df.nunique()

app_id                                 315
app_name                               315
review_id                         21612444
language                                28
timestamp_created                 18932372
timestamp_updated                 18739831
recommended                              2
votes_helpful                         2716
votes_funny                           2215
weighted_vote_score                1304322
comment_count                          351
steam_purchase                           2
received_for_free                        2
written_during_early_access              2
author.steamid                    12406560
author.num_games_owned                7097
author.num_reviews                     983
author.playtime_forever             297891
author.playtime_last_two_weeks       17261
author.playtime_at_review           218306
author.last_played                16009725
dtype: int64

---
Let's check if there's null values in any attribute, and how many if there are.

In [14]:
df.isnull().sum()

app_id                                0
app_name                              0
review_id                             0
language                              0
timestamp_created                     0
timestamp_updated                     0
recommended                           0
votes_helpful                         0
votes_funny                           0
weighted_vote_score                   0
comment_count                         0
steam_purchase                        0
received_for_free                     0
written_during_early_access           0
author.steamid                        0
author.num_games_owned                0
author.num_reviews                    0
author.playtime_forever               2
author.playtime_last_two_weeks        2
author.playtime_at_review         25682
author.last_played                    2
dtype: int64

---
Let's get an idea about the datatype of each attribute.

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21747371 entries, 0 to 21747370
Data columns (total 21 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   app_id                          int64  
 1   app_name                        object 
 2   review_id                       int64  
 3   language                        object 
 4   timestamp_created               int64  
 5   timestamp_updated               int64  
 6   recommended                     bool   
 7   votes_helpful                   int64  
 8   votes_funny                     int64  
 9   weighted_vote_score             float64
 10  comment_count                   int64  
 11  steam_purchase                  bool   
 12  received_for_free               bool   
 13  written_during_early_access     bool   
 14  author.steamid                  int64  
 15  author.num_games_owned          int64  
 16  author.num_reviews              int64  
 17  author.playtime_forever  

---
Let's get an idea about the statistical description of each numerical column (Before Cleaning).

In [16]:
df.describe()

,app_id,review_id,timestamp_created,timestamp_updated,votes_helpful,votes_funny,weighted_vote_score,comment_count,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
count,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.174737e+07,2.172169e+07,2.174737e+07
mean,3.928181e+05,5.187500e+07,1.544432e+09,1.547556e+09,4.044689e+05,1.267917e+05,1.654424e-01,1.308768e-01,7.656120e+16,1.011300e+06,4.044775e+05,1.609105e+04,1.555421e+02,8.807421e+03,1.580505e+09
std,2.480977e+05,2.084267e+07,5.812194e+07,5.689899e+07,1.333741e+09,2.333553e+07,2.434006e-01,2.199398e+00,3.179438e+08,2.108829e+09,1.333741e+09,3.743057e+04,7.300488e+02,2.388553e+04,4.676178e+07
min,7.000000e+01,4.300000e+01,1.290198e+09,1.290198e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.656120e+16,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,2.427600e+05,3.639355e+07,1.510223e+09,1.511729e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.656120e+16,2.200000e+01,2.000000e+00,1.250000e+03,0.000000e+00,5.590000e+02,1.573329e+09
50%,3.595500e+05,5.384058e+07,1.562446e+09,1.572189e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.656120e+16,6.100000e+01,4.000000e+00,4.307000e+03,0.000000e+00,1.881000e+03,1.599058e+09
75%,5.780800e+05,6.928793e+07,1.589696e+09,1.591403e+09,1.000000e+00,0.000000e+00,4.827586e-01,0.000000e+00,7.656120e+16,1.450000e+02,1.000000e+01,1.491200e+04,0.000000e+00,6.823000e+03,1.609002e+09
max,1.291340e+06,8.521867e+07,1.611426e+09,2.283383e+09,4.398047e+12,4.294967e+09,9.959868e-01,4.893000e+03,7.656120e+16,4.398047e+12,4.398047e+12,3.744943e+06,2.703900e+04,3.228103e+06,1.611434e+09


---
---

<a id='Assess'></a>
### b) Assessing
<a id='Quality'></a>
#### i) Quality Issues
Check the columns that I will work on, and check the values range, uniqueness, and validation.
* timestamp_created, timestamp_updated, and author.last_played attributes are integers not datetime.
* There are 134927 duplicated entries.


<a id='Tidiness'></a>
#### ii) Tidiness Issues
* The author attributes describe the author variables which are different than the review attributes.

<a id='Clean'></a>
### c) Cleaning
<a id='Define'></a>
#### i) Define
* Convert timestamp_created, timestamp_updated, and author.last_played attributes to datetime type.
* Drop the duplicated entries.
* Create a separate table for author attributes.

<a id='Code'></a>
#### ii) Code

In [17]:
df_cleaned = df.copy()

Convert timestamp_created, timestamp_updated, and author.last_played attributes to datetime type.

In [18]:
dates_attributes = ['timestamp_created', 'timestamp_updated', 'author.last_played']
for i in dates_attributes:
    df_cleaned[i] = pd.to_datetime(df_cleaned[i], origin='unix', unit='s')

Drop the duplicated entries.

In [19]:
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

Create a separate table for author attributes.

In [20]:
df_cleaned.columns

Index(['app_id', 'app_name', 'review_id', 'language', 'timestamp_created',
       'timestamp_updated', 'recommended', 'votes_helpful', 'votes_funny',
       'weighted_vote_score', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access', 'author.steamid',
       'author.num_games_owned', 'author.num_reviews',
       'author.playtime_forever', 'author.playtime_last_two_weeks',
       'author.playtime_at_review', 'author.last_played'],
      dtype='object')

In [21]:
df_reviews = df_cleaned[df_cleaned.columns[:-7]].copy()

In [22]:
df_author_cols = [df_cleaned.columns[2]]
df_author_cols.extend(list(df_cleaned[df_cleaned.columns[-7:]].columns))

df_author = df_cleaned[df_author_cols].copy()

<a id='Test'></a>
#### iii) Test

Testing the converted date attributes after converting them to datetime.

In [23]:
for i in dates_attributes:
    print('Attribute Name: {} \tEntry[0]: {}'.format(i, df_cleaned[i][0]))

Attribute Name: timestamp_created 	Entry[0]: 2021-01-23 06:00:29
Attribute Name: timestamp_updated 	Entry[0]: 2021-01-23 06:00:29
Attribute Name: author.last_played 	Entry[0]: 2021-01-22 19:23:03


In [24]:
df_cleaned[dates_attributes].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21612444 entries, 0 to 21612443
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   timestamp_created   datetime64[ns]
 1   timestamp_updated   datetime64[ns]
 2   author.last_played  datetime64[ns]
dtypes: datetime64[ns](3)
memory usage: 494.7 MB


In [25]:
df_reviews[dates_attributes[:-1]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21612444 entries, 0 to 21612443
Data columns (total 2 columns):
 #   Column             Dtype         
---  ------             -----         
 0   timestamp_created  datetime64[ns]
 1   timestamp_updated  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 329.8 MB


In [26]:
df_author[[dates_attributes[-1]]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21612444 entries, 0 to 21612443
Data columns (total 1 columns):
 #   Column              Dtype         
---  ------              -----         
 0   author.last_played  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 164.9 MB


Testing the number of duplicated entries after dropping them.

In [27]:
df_cleaned.duplicated().sum()

0

Displaying the new 2 dataframes.
1. reviews dataframe.

In [28]:
display(df_reviews.shape)
display(df_reviews.head(2))
df_reviews.tail(2)

(21612444, 14)

,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,2021-01-23 06:00:29,2021-01-23 06:00:29,True,0,0,0.0,0,True,False,False
1,292030,The Witcher 3: Wild Hunt,85185250,schinese,2021-01-23 05:50:30,2021-01-23 05:50:30,True,0,0,0.0,0,True,False,False


,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
21612442,546560,Half-Life: Alyx,65645066,english,2020-03-23 18:03:51,2020-04-08 21:47:02,True,0,0,0.386022,11,True,False,False
21612443,546560,Half-Life: Alyx,65644930,schinese,2020-03-23 18:01:45,2020-06-13 15:06:40,True,0,0,0.416107,5,True,False,False


2. author dataframe.

In [29]:
display(df_author.shape)
display(df_author.head(2))
df_author.tail(2)

(21612444, 8)

,review_id,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,85185598,76561199095369542,6,2,1909.0,1448.0,1909.0,2021-01-22 19:23:03
1,85185250,76561198949504115,30,10,2764.0,2743.0,2674.0,2021-01-23 07:18:27


,review_id,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
21612442,65645066,76561198041763187,140,3,210.0,0.0,12.0,2020-05-17 11:36:55
21612443,65644930,76561198116747069,253,7,1062.0,0.0,8.0,2020-10-16 14:12:23


Exporting the cleaned dataframes, so we don't have to run the whole cleaning process again each time.

In [30]:
df_cleaned.to_csv('df_cleaned.csv', index=False)
df_reviews.to_csv('df_reviews.csv', index=False)
df_author.to_csv('df_author.csv', index=False)

Importing the exported dataframes and making sure to import the datetime attributes as datetime type not string.

In [31]:
dates_attributes = ['timestamp_created', 'timestamp_updated', 'author.last_played']

#df_cleaned = pd.read_csv('df_cleaned.csv', parse_dates=dates_attributes)
df_reviews = pd.read_csv('df_reviews.csv', parse_dates=dates_attributes[:-1])
#df_author = pd.read_csv('df_author.csv', parse_dates=[dates_attributes[-1]])

Check that the loaded date attributes in our cleaned dataset is loaded as datetime without any problem.

In [32]:
df_reviews[dates_attributes[:-1]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21612444 entries, 0 to 21612443
Data columns (total 2 columns):
 #   Column             Dtype         
---  ------             -----         
 0   timestamp_created  datetime64[ns]
 1   timestamp_updated  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 329.8 MB


Since, everything went well as we planned, it's time for EDA. 
There is a minor issue which is there are null values in two attributes (author.playtime_at_review & author.last_played) so we will handle them only in case we need to include them in our analysis.